# Context

- Using hdf5 compressed file as input and splitting it into hdf5 datasets (in 1 hdf5 file but physically chunked).
- We remarked that the reading time seemed extremely fast compared to the writing time.

## Experiment 2.0.1 - Writing from hdf5 to numpy files (`stack`):

Experiment setting: 
- one buffer (= 2.5GB)
- one thread
- [ ] faire tourner ces tests sur bluff
- [ ] faire plusieurs runs pour voir la variabilité

<img src="images/to_npy_stack_1thread.png" width="500">

Observations:
- reading time speed >> writing time speed
- 2.5GB of data have been effectively loaded so the data should be uncompressed.
- read time for 1 buffer: 1s to 2s 
- write time for 1 buffer: ~20s 

**Side note:** Using 1 thread here. We figured out that as the disk is sequential there is no need for us to use more threads. The memory constraint was already constraining the scheduler to run one loading task at a time but we missed the fact that the writing was still parallel, hence making seeks while writing several buffers at the same time:

<img src="images/to_npy_stack.png" width="500">

## Questions
- Why is the reading time so fast compared to the writing time?

## Experiment 2.0.2 - What is the normal read/write speed of the computer I am working on?
Methodology:
**TODO**
  
  
- [ ] faire tourner ces tests sur bluff
- [ ] faire plusieurs runs pour voir la variabilité

Output: 
**TODO**

## Experiment 2.1 - Does the write time contain the decompression?
1. What is the real writing time?
2. Does the sum read time + decompression time ~= 1s to 2s hold?

### Experiment 2.1.1 - What is the real writing time?
**Methology:**
- creating an **uncompressed** random array of 2.5GB (2.5GB = size of 1 buffer in experiment `small`) 
- measuring the time needed to write it in a numpy file 
    - using `da.to_stack` dask function
    - using `np.save` numpy function

  
  
- [ ] faire tourner ces tests sur bluff
- [ ] faire plusieurs runs pour voir la variabilité

**Output of function:**
- da.to_stack: **TODO**
- np.save: The writing time is ~18.8s

**Conclusion:**
- The writing time in this experiment matches the writing time in the `Experiment 2.0.1` so the decompression does not seem to be included in the write time.

### Experiment 2.1.2 - What is the decompression time and the time for reading the compressed file?
**Methology:**
- MEASURE:
    - time to read compressed file
    - time to uncompress file in memory
- COMPARE:
    - **if** time to read + time to uncompress ~= 1 to 2s (time to read in previous experiment) **then** the write time does not seem to contain decompression.
    - to confirm this however, we need to see if the time to write uncompressed data alone is really about 18.8s (`Experiment 2.1.1` above) 


- [ ] faire tourner ces tests sur bluff
- [ ] faire plusieurs runs pour voir la variabilité

**Output of function:**

**Conclusion:**


**Next step:**
The reading time is **small** but the writing time seems normal according to previous experiments. Therefore, we want to understand what makes the reading time so small?

**Question**: Why is the reading time so small?

# Experiment 2.2 - Why is the reading time so small?
**Hypotheses**:

The reading time seems to be small because of two factors:
1. file is compressed -> reading is fast
2. file contains mostly zeros -> decompression is fast
    - because file contains random data drawn from Normal(0,1) distribution

## Experiment 2.2.1: Reading is fast because the file is compressed. 
**Hypothesis:** Reading is fast because the file is compressed. 
    
**Methodology:**
0. context:
    - create a 2.5GB compressed hdf5 filedrawn from normal(0,1) distribution
    - create a 2.5GB uncompressed hdf5 file drawn from normal(0,1) distribution
    
1. measure:
    - read time for compressed file
    - read time for uncompressed file

2. compare: 
    - **If** compressed_file_readtime << uncompressed_file_readtime **then,** hypothesis seems validated.
    - **If** uncompressed file_readtime ~= reading time for 2.5GB of data **then** hypothesis seems validated.
  
  
- [ ] faire tourner ces tests sur bluff
- [ ] faire plusieurs runs pour voir la variabilité

**Output of function:**

    
**Next step:**
    


## Experiment 2.2.2: Decompression is fast because of the normal distribution.
**Hypothesis:** Decompression is fast because it has been drawn from a normal(0,1) distribution.
    
**Methodology:**
0. context:
    - create a 2.5GB **compressed** hdf5 filedrawn from **normal**(0,1) distribution
    - create a 2.5GB **compressed** hdf5 file drawn from **uniform** distribution
1. measure:
    - time to read file in main memory
    - decompression time
    
    - -> see if it sums to 1~2 seconds for the normal(0,1)
    - -> see if the decompression time is bigger for the uniform distribution
  
  
- [ ] faire tourner ces tests sur bluff
- [ ] faire plusieurs runs pour voir la variabilité
    
**Output of function:**
- worst case time to load compressed file: 0.28s 
- worst case time to load data in RAM: 0.8s 
- Overall time to load data in memory from compressed file ~= 1s


- saving time to numpy stack or npy file is almost the same
    - 0.8s + 18.8s = 19.6s (npy file)
    - 19.1s (npy stack)
- gzip-based hdf5 file write time:~16s
- uncompressed hdf5: ~82s
    
**Next step:**


# Lossless compression filters

For h5py `group` used in `datasets`

- **GZIP filter ("gzip")**
    Available with every installation of HDF5, so it’s best where portability is required. Good compression, moderate speed. compression_opts sets the compression level and may be an integer from 0 to 9, default is 4.
- **LZF filter ("lzf")**
    Available with every installation of h5py (C source code also available). Low to moderate compression, very fast. No options.
- **SZIP filter ("szip")**
    Patent-encumbered filter used in the NASA community. Not available with all installations of HDF5 due to legal reasons. Consult the HDF5 docs for filter options.

# On the impact of hdf5 compression on clustered reads/writes
gzip requiert de lire toutes les données pour compresser => comment fait dask alors puisquil ecrit en "parallele"?
-> on ne va pas perdre en efficacité avec clustered strategy even if gzip parce que la rapidité de dask justement avec des petits buffers cest parce qu'il store en chunked donc il nous suffit de store en chunk physique aussi avec chunk size = buffer size. Peut etre meme quon peut paralléliser la compression pour que ca aille plus vite.

si on veut rechunk ca ne marche pas

# Questions à traiter:
- estce que gzip est indexé? lire la these de yongping et le papier de recherche
- faire tests sans compression pour ne pas avoir de variabilité selon de bloc que l'on choisi 